In [1]:
%matplotlib inline


from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt

#import k-nn classifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import operator


iris = datasets.load_iris()

#view a description of the dataset (uncomment next line to do so)
#print(iris.DESCR)

#Set X equal to features, Y equal to the targets

X=iris.data 
y=iris.target 


mySeed=1234567
#initialize random seed generator 
np.random.seed(mySeed)

#we add some random noise to our data to make the task more challenging
X=X+np.random.normal(0,0.5,X.shape)

In [2]:
# nested cross validation function
# X - data / features
# y - outputs
# foldK - number of folds
# nns - list of number of neighbours parameter for validation
# dists - list of distances for validation
# mySeed - random seed
# returns: accuracy over 5 folds (list)

def knnise(training,labels,test,neighbours,myMetric):
    knn=KNeighborsClassifier(n_neighbors=neighbours, metric=myMetric)
    
    #define training and testing data, fit the classifier
    knn.fit(training,labels)
    
    #predict values for test data based on training data
    k_non=knn.predict(test)
    return k_non;

def myAccuracy(testing,predicted):
    mistakes=0
    for i in range (len(testing)):
        if (testing[i]!=predicted[i]): mistakes+=1
    return 1-mistakes/len(testing);

def myNestedCrossVal(X,y,foldK,nns,dists,mySeed):
    np.random.seed(mySeed)
    accuracy_fold=np.zeros(foldK)
    
    #TASK: use the function np.random.permutation to generate a list of shuffled indices from in the range (0,number of data)
    indices=np.random.permutation(np.arange(len(X)))
    
    #TASK: use the function array_split to split the indices to foldK different bins (here, 5)
    bins=np.split(indices,foldK)
    
    #no need to worry about this, just checking that everything is OK
    assert(foldK==len(bins))
    
    #loop through folds
    for foldNum in range(0,foldK):
        # list to save current indices for testing
        foldTest=bins[foldNum%foldK]  
        
        # list to save current indices for validation
        foldVal=bins[(foldNum+1)%foldK]    
        
        #loop through all bins, take bin i for testing, the next bin for validation and the rest for testing
        # list to save current indices for training
        foldTrain=np.delete(bins,[foldNum%foldK,(foldNum+1)%foldK],0).flatten()    
        
        #no need to worry about this, just checking that everything is OK
        assert not np.intersect1d(foldTest,foldVal)
        assert not np.intersect1d(foldTrain,foldTest)
        assert not np.intersect1d(foldTrain,foldVal)
       
        #save the best distance metric here
        bestDistance='' 
        #save the best number of neighbours here
        bestNN=-1 
        #save the best attained accuracy here (in terms of validation) 
        bestAccuracy=-10 
        
        # loop through all parameters (one for loop for distances, one for loop for nn)
        for distLoop in range (0,len(dists)):
            for neighLoop in range (0,len(nns)):
                
                # train the classifier on current number of neighbours/distance
                val_pred=knnise(X[foldTrain],y[foldTrain],X[foldVal],nns[neighLoop],dists[distLoop])
                
                # obtain results on validation 
                currentAccuracy=myAccuracy(y[foldVal],val_pred)
                
                # save parameters if results are the best we had
                if (currentAccuracy>bestAccuracy): 
                    bestAccuracy=currentAccuracy
                    bestDistance=dists[distLoop] 
                    bestNN=nns[neighLoop]
        print('** End of val for this fold, best NN', bestNN, 'best Dist', bestDistance)
        #'''
        
        #evaluate on test data:
        #extend your training set by including the validation set             
        foldTrain=np.concatenate((foldTrain,foldVal),0)
        #train k-NN classifier on new training set and test on test set
        test_pred=knnise(X[foldTrain],y[foldTrain],X[foldTest],bestNN,bestDistance)
        #get performance on fold, save result in accuracy_fold array
        accuracy_fold[foldNum]=myAccuracy(y[foldTest],test_pred)
        
        print('==== Final Cross-val on test on this fold with NN', bestNN, 'dist', bestDistance, ' accuracy ',accuracy_fold[foldNum])
    return accuracy_fold;
    
#call your nested crossvalidation function:
accuracy_fold=myNestedCrossVal(X,y,5,list(range(1,11)),['euclidean','manhattan'],mySeed)

print(accuracy_fold)
print(np.mean(accuracy_fold))
print(np.std((accuracy_fold)))

** End of val for this fold, best NN 1 best Dist euclidean
==== Final Cross-val on test on this fold with NN 1 dist euclidean  accuracy  0.733333333333
** End of val for this fold, best NN 9 best Dist euclidean
==== Final Cross-val on test on this fold with NN 9 dist euclidean  accuracy  0.833333333333
** End of val for this fold, best NN 2 best Dist euclidean
==== Final Cross-val on test on this fold with NN 2 dist euclidean  accuracy  0.8
** End of val for this fold, best NN 5 best Dist euclidean
==== Final Cross-val on test on this fold with NN 5 dist euclidean  accuracy  1.0
** End of val for this fold, best NN 8 best Dist euclidean
==== Final Cross-val on test on this fold with NN 8 dist euclidean  accuracy  0.866666666667
[ 0.73333333  0.83333333  0.8         1.          0.86666667]
0.846666666667
0.0884433277428


In [4]:
np.random.seed(mySeed)
indices= np.random.permutation(X.shape[0]) 
bins=np.array_split(indices,2) # we  just need a training and testing set here
foldTrain=bins[0]
foldTest=bins[1]

knn=KNeighborsClassifier(n_neighbors=10, metric='euclidean')
knn.fit(X[foldTrain],y[foldTrain])
y_pred=knn.predict(X[foldTest])
print(accuracy_score(y[foldTest],y_pred))

## ANSWER HERE: Suggested code structure in comments below
# given a test point, your code should
# 
# - get the 'nearest neighbours' - i.e. the samples in the training set - that are nearest to our test sample
# -----> done by evaluating the distance of the test sample to all samples in the training set
# - assign a label to the test sample based on the 'neighbours'

##=== FUNCTION DEFINITIONS  ===##

#define distance functions: given two vectors (ndarrays), this function returns the distance between them
#Write at least two distance functions, measuring the squared distance between your data and the absolute value distance.
def euclideanVectors(in1,in2):
    return np.sqrt(np.sum(np.square(in1-in2)));
def manhattanVectors(in1,in2):
    return np.sum(np.absolute(in1-in2));
#You can implement both of these by looking at the numpy.linalg.norm method, or implement your own version.  

#The get neighbours function  returns the nearest neighbour indices in X of the test point x_.  In more detail
# Input: x_ : point in test data
#       X   : training data
#       n   : number of neighbours to return
#       T   : total number of training data
# Output: n-nearest neighbours of x_ in training data X

def getNeighbours(testing,training,neighbours,metric): # where T is number of data
    distances=np.empty(0)
    if (metric=='euclidean'): 
        for i in range (0,training.shape[0]):
            distances=np.append(distances,euclideanVectors(training[i],testing))
    if (metric=='manhattan'): 
        for i in range (0,training.shape[0]):
            distances=np.append(distances,manhattanVectors(training[i],testing))
    indices=np.argsort(distances)

    print ("Predicted Neighbour distances:",distances)
   
    if (neighbours>indices.shape[0]): 
        neighbours=indices.shape[0]
    closest=np.zeros(neighbours)
    for n in range (0,neighbours):
        closest[n]=indices[n]
    print ("Indexes of predicted neighbours: ",closest)
    #based on misunderstainding of argsort
    '''
    ngb=0
    for n in range (0,len(indices)):
        if (indices[n]<neighbours):
            closest[ngb]=n
            ngb+=1
            if (ngb==neighbours): 
                break
    '''
    return closest# indices of n-nearest neighbours in training data

# The assign label function returns the assigned label for a test data point, given the labels of nearest neighbours
# Input: nLabels : labels (classes) of nearest neighbours of a test point
# Output: the assigned label
# e.g., if we have n=1 (one neighbour), then we can just return the label of the nearest neighbour
# else, we can e.g., choose the majority class
def assignLabel(nLabels):
    #labels=np.zeros(len(np.unique(nLabels))) this throws an error when labels are 1 and 2 only
    highestLabel=int(np.unique(nLabels).max())+1
    labels=np.zeros(highestLabel)
    for labelLoop in range (0,highestLabel):
        labels[int(nLabels[labelLoop])]+=1
    bestLabel=0;
    labelIndex=0;
    for labelCountLoop in range (0,len(labels)):
        if (labels[labelCountLoop]>bestLabel):
            bestLabel=labels[labelCountLoop]
            labelIndex=labelCountLoop
    return labelIndex;# label assigned to test point x_
'''
infoTest=np.array([3,8,4,2])
infoTrain=np.array([[3,4,4,2],[3,6,1,2],[3,8,4,2],[3,10,2,4]])
print(getNeighbours(infoTest,infoTrain,2,'euclidean'))

print(assignLabel(np.array([0,1,2,2,3,3,2,2])))
'''


# here is some sample code for evaluating the kNN classifier you just built
# NOTE: this is just a suggested way to do this - you can do it in another way if you want
correct=0;
metric='euclidean'
NN=10
for i in foldTest: #for all test points
    # knn classifier
    x_=X[i] # test point x_
    y_=y[i] # true label for y_
    print ('Test point: ',x_)
    print ('True label: ',y_)
    knn=NearestNeighbors(n_neighbors=10, metric='euclidean')
    knn.fit(X[foldTrain])
    print('Actual neihgbour indexes in X: ',knn.kneighbors(x_.reshape(1,-1),return_distance=0)[0])
    print('Actual neihgbour distances in X: ',knn.kneighbors(x_.reshape(1,-1))[0])
    
    # get neighbours of x_ in training data 
    tempTrain=np.zeros((len(foldTrain),4))
    for j in range (0,len(foldTrain)):
        tempTrain[j]=X[foldTrain[j]]
    neighbs=getNeighbours(x_,tempTrain,NN,metric)
    print('Predicted neighbour indexes in foldTrain: ',neighbs)
    print('Predicted neighbour distances in foldTrain: ')
    for b in range (0, len(neighbs)):
        print(euclideanVectors(X[foldTrain[int(neighbs[b])]],x_))
    
    # assignLabel to x_ based on neighbours
    neighbourLabels=np.zeros(NN)
    for tempLabel in range (0,NN):
        neighbourLabels[tempLabel]=y[foldTrain[int(neighbs[tempLabel])]]
    label=assignLabel(neighbourLabels)
    print('Neighbour labels: ', neighbourLabels)
    print('Predicted label: ', label)
    print(' ')
    # evaluate if the assigned label is correct (equal to y_)
    if (label==y_):
        correct+=1
    
print (correct/len(foldTest))

0.88
Test point:  [ 5.0515576   3.78397378  2.33417435  0.71110957]
True label:  0
Actual neihgbour indexes in X:  [13 43 52  5 19 48 22 60  9 31]
Actual neihgbour distances in X:  [[ 0.79095231  0.80715889  0.96314091  0.98774615  1.05130103  1.07234694
   1.11675755  1.11909147  1.13744226  1.35627128]]
Predicted Neighbour distances: [ 2.83273045  2.90539899  2.81065017  2.73399039  3.39224403  0.98774615
  2.61489577  3.41497647  1.55236842  1.13744226  3.53200328  5.22277512
  5.98209261  0.79095231  1.8341989   3.1451558   2.13263249  3.57131119
  1.9780232   1.05130103  2.81666932  2.66730043  1.11675755  1.68676853
  1.8393682   2.30112424  3.13526411  2.64355489  3.9557224   3.90020921
  4.28860728  1.35627128  2.47006672  2.68026157  1.83211937  2.55983386
  1.40651215  2.5768173   1.95422401  4.03635856  5.5840142   2.45435828
  2.83799739  0.80715889  1.4716739   2.81782716  2.77025012  1.88039233
  1.07234694  1.71252079  2.48583962  2.29242524  0.96314091  3.70246245
  3.1

Indexes of predicted neighbours:  [ 67.   6.  72.  26.   1.   0.  63.  54.  74.  33.]
Predicted neighbour indexes in foldTrain:  [ 67.   6.  72.  26.   1.   0.  63.  54.  74.  33.]
Predicted neighbour distances in foldTrain: 
0.990402119746
1.02992507319
1.03622695396
1.0930756213
1.22658818327
1.32965724131
1.40578683006
1.48969793108
1.49148882614
1.58539186208
Neighbour labels:  [ 2.  1.  1.  1.  1.  2.  1.  1.  1.  2.]
Predicted label:  1
 
Test point:  [ 4.59800495  3.91807515  1.33170173  0.20031234]
True label:  0
Actual neihgbour indexes in X:  [19 49  9 55 22  8 48 18 13 60]
Actual neihgbour distances in X:  [[ 0.45636899  0.78566961  0.84236365  0.86477809  0.98676509  1.05740757
   1.08206042  1.08210002  1.15459688  1.17811117]]
Predicted Neighbour distances: [ 3.98876568  4.080939    2.49775824  3.8671359   4.4471779   1.19111156
  3.80717568  4.5884748   1.05740757  0.84236365  4.63005333  6.43612917
  7.18582049  1.15459688  2.95165459  4.3167705   1.56091017  4.67753572

In [5]:
# nested cross validation function + KNN
# X - data / features
# y - outputs
# foldK - number of folds
# nns - list of number of neighbours parameter for validation
# dists - list of distances for validation
# mySeed - random seed
# returns: accuracy over 5 folds (list)
'''
def knnise(training,labels,test,neighbours,myMetric):
    knn=KNeighborsClassifier(n_neighbors=neighbours, metric=myMetric)
    
    #define training and testing data, fit the classifier
    knn.fit(training,labels)
    
    #predict values for test data based on training data
    k_non=knn.predict(test)
    return k_non;
'''
def myAccuracy(testing,predicted):
    mistakes=0
    for i in range (len(testing)):
        if (testing[i]!=predicted[i]): mistakes+=1
    return 1-mistakes/len(testing);

def euclideanVectors(in1,in2):
    return np.sqrt(np.sum(np.square(in1-in2)));
def manhattanVectors(in1,in2):
    return np.sum(np.absolute(in1-in2));
#You can implement both of these by looking at the numpy.linalg.norm method, or implement your own version.  

#The get neighbours function  returns the nearest neighbour indices in X of the test point x_.  In more detail
# Input: x_ : point in test data
#       X   : training data
#       n   : number of neighbours to return
#       T   : total number of training data
# Output: n-nearest neighbours of x_ in training data X

def getNeighbours(testing,training,neighbours,metric): # where T is number of data
    distances=np.empty(0)
    if (metric=='euclidean'): 
        for i in range (0,training.shape[0]):
            distances=np.append(distances,euclideanVectors(training[i],testing))
    if (metric=='manhattan'): 
        for i in range (0,training.shape[0]):
            distances=np.append(distances,manhattanVectors(training[i],testing))
    indices=np.argsort(distances)

    print ("Predicted Neighbour distances:",distances)
   
    if (neighbours>indices.shape[0]): 
        neighbours=indices.shape[0]
    closest=np.zeros(neighbours)
    for n in range (0,neighbours):
        closest[n]=indices[n]
    print ("Indexes of predicted neighbours: ",closest)
    #based on misunderstainding of argsort
    '''
    ngb=0
    for n in range (0,len(indices)):
        if (indices[n]<neighbours):
            closest[ngb]=n
            ngb+=1
            if (ngb==neighbours): 
                break
    '''
    return closest# indices of n-nearest neighbours in training data

# The assign label function returns the assigned label for a test data point, given the labels of nearest neighbours
# Input: nLabels : labels (classes) of nearest neighbours of a test point
# Output: the assigned label
# e.g., if we have n=1 (one neighbour), then we can just return the label of the nearest neighbour
# else, we can e.g., choose the majority class
def assignLabel(nLabels):
    #labels=np.zeros(len(np.unique(nLabels))) this throws an error when labels are 1 and 2 only
    highestLabel=int(np.unique(nLabels).max())+1
    labels=np.zeros(highestLabel)
    for labelLoop in range (0,highestLabel):
        labels[int(nLabels[labelLoop])]+=1
    bestLabel=0;
    labelIndex=0;
    for labelCountLoop in range (0,len(labels)):
        if (labels[labelCountLoop]>bestLabel):
            bestLabel=labels[labelCountLoop]
            labelIndex=labelCountLoop
    return labelIndex;# label assigned to test point x_

# here is some sample code for evaluating the kNN classifier you just built
# NOTE: this is just a suggested way to do this - you can do it in another way if you want

def knnise(neighbours,myMetric):
    correct=0;
    for i in foldTest: #for all test points
        # knn classifier
        x_=test # test point x_
        y_=label # true label for y_
    
        # get neighbours of x_ in training data 
        tempTrain=np.zeros((len(foldTrain),4))
        for j in range (0,len(foldTrain)):
            tempTrain[j]=X[foldTrain[j]]
        neighbs=getNeighbours(x_,tempTrain,neighbours,myMetric)

        # assignLabel to x_ based on neighbours
        neighbourLabels=np.zeros(neighbours)
        for tempLabel in range (0,neighbours):
            neighbourLabels[tempLabel]=y[foldTrain[int(neighbs[tempLabel])]]
        label=assignLabel(neighbourLabels)

        # evaluate if the assigned label is correct (equal to y_)
        if (label==y_):
            correct+=1
    return (correct/len(foldTest))

def myNestedCrossVal(X,y,foldK,nns,dists,mySeed):
    np.random.seed(mySeed)
    accuracy_fold=np.zeros(foldK)
    

    indices=np.random.permutation(np.arange(len(X)))
    
    bins=np.split(indices,foldK)
    
    #loop through folds
    for foldNum in range(0,foldK):
        # list to save current indices for testing
        foldTest=bins[foldNum%foldK]  
        
        # list to save current indices for validation
        foldVal=bins[(foldNum+1)%foldK]    
        
        #loop through all bins, take bin i for testing, the next bin for validation and the rest for testing
        # list to save current indices for training
        foldTrain=np.delete(bins,[foldNum%foldK,(foldNum+1)%foldK],0).flatten()    
        
        #no need to worry about this, just checking that everything is OK
        assert not np.intersect1d(foldTest,foldVal)
        assert not np.intersect1d(foldTrain,foldTest)
        assert not np.intersect1d(foldTrain,foldVal)
       
        #save the best distance metric here
        bestDistance='' 
        #save the best number of neighbours here
        bestNN=-1 
        #save the best attained accuracy here (in terms of validation) 
        bestAccuracy=-10 
        
        # loop through all parameters (one for loop for distances, one for loop for nn)
        for distLoop in range (0,len(dists)):
            for neighLoop in range (0,len(nns)):
                
                # train the classifier on current number of neighbours/distance
                val_pred=knnise(X[foldTrain],y[foldTrain],X[foldVal],nns[neighLoop],dists[distLoop])
                
                # obtain results on validation 
                currentAccuracy=myAccuracy(y[foldVal],val_pred)
                
                # save parameters if results are the best we had
                if (currentAccuracy>bestAccuracy): 
                    bestAccuracy=currentAccuracy
                    bestDistance=dists[distLoop] 
                    bestNN=nns[neighLoop]
        print('** End of val for this fold, best NN', bestNN, 'best Dist', bestDistance)
        #'''
        
        #evaluate on test data:
        #extend your training set by including the validation set             
        foldTrain=np.concatenate((foldTrain,foldVal),0)
        #train k-NN classifier on new training set and test on test set
        test_pred=knnise(X[foldTrain],y[foldTrain],X[foldTest],bestNN,bestDistance)
        #get performance on fold, save result in accuracy_fold array
        accuracy_fold[foldNum]=myAccuracy(y[foldTest],test_pred)
        
        print('==== Final Cross-val on test on this fold with NN', bestNN, 'dist', bestDistance, ' accuracy ',accuracy_fold[foldNum])
    return accuracy_fold;
    
#call your nested crossvalidation function:
accuracy_fold=myNestedCrossVal(X,y,5,list(range(1,11)),['euclidean','manhattan'],mySeed)

print(accuracy_fold)
print(np.mean(accuracy_fold))
print(np.std((accuracy_fold)))


UnboundLocalError: local variable 'label' referenced before assignment